<br>
<br>

# **준비 사항**

In [47]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers --quiet

In [23]:
import tensorflow as tf
import torch

from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import time
import datetime
import json
import os
import re

<br>
<br>

# **데이터 로드**

In [5]:
from google.colab import drive
drive.mount('content')

Mounted at content


In [43]:
# friends dataset 탐색
root_dir = "content"

for (root, dirs, files) in os.walk(root_dir):
    for file in files:
        if file.lower().startswith('friends') and file.lower().endswith('.json'):
            filepath = root
            
print(filepath)

# JSON -> DataFrame
data = {'train': {'speaker': [], 'utterance': [], 'emotion': []},
        'dev': {'speaker': [], 'utterance': [], 'emotion': []},
        'test': {'speaker': [], 'utterance': [], 'emotion': []}}

for dtype in ['train', 'dev', 'test']:
    for dialog in json.loads(open(os.path.join(filepath, 'friends_' + dtype + '.json')).read()):
        for line in dialog:
            data[dtype]['speaker'].append(line['speaker'])
            data[dtype]['utterance'].append(line['utterance'])
            data[dtype]['emotion'].append(line['emotion'])

train = pd.DataFrame(data['train'])
dev = pd.DataFrame(data['dev'])
test = pd.DataFrame(data['test'])

In [46]:
print(train.shape)
print(dev.shape)
print(test.shape)
train.head(10)

(10561, 3)
(1178, 3)
(2764, 3)


,speaker,utterance,emotion
0,Mark,Why do all youre coffee mugs have numbers on ...,surprise
1,Rachel,Oh. Thats so Monica can keep track. That way ...,non-neutral
2,Rachel,Y'know what?,neutral
3,Ross,It didnt.,neutral
4,Frank,"Okay, so what you used to have with Rachel, is...",joy
5,Joey,"Now, wh-what, what is that like?",surprise
6,Frank,"Its so cool man, its so, its just cause be...",joy
7,Ross,"Yeah, yeah.",neutral
8,Joey,Why cant I find that?,non-neutral
9,Ross,"Dont ask me, I had it and I blew it!",anger


In [ ]:
#  모든 데이터셋을 통틀어 가장 긴 토큰 길이 확인
temp_df = pd.concat([train, dev, test])

length = 0
max_length = 0

for text in temp_df.utterance.values:
    if len(text.split()) > length:
        max_length = len(text.split())
    length = len(text.split())
print(max_length)

12


<br>
<br>

# **전처리 - 훈련셋**

In [ ]:
# 리뷰 문장 추출
sentences = train['utterance']
sentences[:10]

0    also I was the point person on my companys tr...
1                     You mustve had your hands full.
2                              That I did. That I did.
3        So lets talk a little bit about your duties.
4                               My duties?  All right.
5    Now youll be heading a whole division, so you...
6                                               I see.
7    But therell be perhaps 30 people under you so...
8                                        Good to know.
9                                We can go into detail
Name: utterance, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] also I was the point person on my company\x92s transition from the KL-5 to GR-6 system. [SEP]',
 '[CLS] You must\x92ve had your hands full. [SEP]',
 '[CLS] That I did. That I did. [SEP]',
 '[CLS] So let\x92s talk a little bit about your duties. [SEP]',
 '[CLS] My duties?  All right. [SEP]',
 '[CLS] Now you\x92ll be heading a whole division, so you\x92ll have a lot of duties. [SEP]',
 '[CLS] I see. [SEP]',
 '[CLS] But there\x92ll be perhaps 30 people under you so you can dump a certain amount on them. [SEP]',
 '[CLS] Good to know. [SEP]',
 '[CLS] We can go into detail [SEP]']

In [ ]:
# 라벨 추출
encoder = LabelEncoder()
labels = train['emotion'].values
encoder.fit(labels)
labels = encoder.transform(labels)

print(encoder.classes_)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'non-neutral' 'sadness'
 'surprise']


In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-large-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] also I was the point person on my companys transition from the KL-5 to GR-6 system. [SEP]
['[CLS]', 'also', 'I', 'was', 'the', 'point', 'person', 'on', 'my', 'company', '##s', 'transition', 'from', 'the', 'K', '##L', '-', '5', 'to', 'G', '##R', '-', '6', 'system', '.', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 20

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([ 101, 1145,  146, 1108, 1103, 1553, 1825, 1113, 1139, 1419, 1116,
       6468, 1121, 1103,  148, 2162,  118,  126, 1106,  144])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([ 101, 1135,  112,  188, 1136, 6276,  106,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(0)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
tensor([  101,  1337, 18243,  1144,  1400,  1106,  1129,  4482,  1174,   117,
         3008,   117,  1115,  1116,  1240,  2853,  1208,   119,   102,     0])
tensor(4)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0.])


In [ ]:
# 배치 사이즈
batch_size = 64

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

<br>
<br>

# **전처리 - 테스트셋**

In [ ]:
# 리뷰 문장 추출
sentences = test['emotion']
sentences[:10]

0       surprise
1    non-neutral
2        neutral
3        neutral
4            joy
5       surprise
6            joy
7        neutral
8    non-neutral
9          anger
Name: emotion, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] surprise [SEP]',
 '[CLS] non-neutral [SEP]',
 '[CLS] neutral [SEP]',
 '[CLS] neutral [SEP]',
 '[CLS] joy [SEP]',
 '[CLS] surprise [SEP]',
 '[CLS] joy [SEP]',
 '[CLS] neutral [SEP]',
 '[CLS] non-neutral [SEP]',
 '[CLS] anger [SEP]']

In [ ]:
# 라벨 추출
encoder = LabelEncoder()
labels = test['emotion'].values
encoder.fit(labels)
labels = encoder.transform(labels)

print(encoder.classes_)

['anger' 'disgust' 'fear' 'joy' 'neutral' 'non-neutral' 'sadness'
 'surprise']


In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-large-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] surprise [SEP]
['[CLS]', 'surprise', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 20

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([ 101, 3774,  102,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([ 101, 3774,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(7)
tensor([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])


In [ ]:
# 배치 사이즈
batch_size = 64

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

<br>
<br>

# **모델 생성**

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-large-cased", num_labels=8)
model.cuda()

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 5

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

<br>
<br>

# **모델 학습**

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.10
  Training epcoh took: 0:01:19

Running Validation...
  Accuracy: 0.60
  Validation took: 0:00:02

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.10
  Training epcoh took: 0:01:19

Running Validation...
  Accuracy: 0.59
  Validation took: 0:00:02

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.09
  Training epcoh took: 0:01:19

Running Validation...
  Accuracy: 0.58
  Validation took: 0:00:02

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.08
  Training epcoh took: 0:01:19

Running Validation...
  Accuracy: 0.58
  Validation took: 0:00:02

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.07
  Training epcoh took: 0:01:19

Running Validation...
  Accuracy: 0.58
  Validation took: 0:00:02

Training complete!


에폭마다 훈련셋과 검증셋을 반복하여 학습을 수행합니다. 

<br>
<br>

# **테스트셋 평가**

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.72
Test took: 0:00:06


<br>
<br>

# **새로운 문장 테스트**

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
def predict(sentence):
    logits = test_sentences([sentence])
    label = np.argmax(logits)
    emotion = encoder.classes_[label]
    return emotion

print(logits)
print(np.argmax(logits))

[[-1.5722619  3.2715435 -2.461211  -2.2554595 -2.7080073  4.1809144
  -2.2216432  5.141793 ]]
7


In [ ]:
logits = test_sentences(["i'm sorry to hear that"])

print(logits)
print(np.argmax(logits))

[[-2.777257  -1.3429648 -1.6831119 -1.0271511  0.8114235 -0.9242955
   9.615311  -2.1560647]]
6


In [ ]:
torch.save(model, filepath)  # 저장
# model = torch.load('./gdrive/MyDrive/sentiment_analysis/bert_nsmc.pt')  # 로드

In [48]:
os.listdir(filepath)

['ko_data.csv',
 'koELECTRA_nsmc_v2.pt',
 'koELECTRA_nsmc_v3.pt',
 'ELECTRA_friends_multi_classification.ipynb',
 'koELECTRA_nsmc_v4.pt',
 'sample.csv',
 'en_data.csv',
 'BERT_friends.pt',
 'friends_sample.csv',
 'koELECTRA_nsmc_v5.pt',
 'koELECTRA_nsmc_10epochs.pt',
 'nsmc_sample.csv',
 'KoELECTRA_nsmc_sentiment_analysis.ipynb',
 'BERT_friends_sentiment_analysis.ipynb',
 'EmotionLines_friends_annotation.tar.gz',
 'friends_dev.json',
 'friends_test.json',
 'friends_train.json']

</br>
</br>

### kaggle 경진대회 dataset 활용 예측

In [ ]:
# 케글 predict 평가용 데이터 로드
kaggle_dataset = pd.read_csv(os.path.join(filepath, 'en_data.csv'))
kaggle_dataset.head()
len(kaggle_dataset)

1623

In [ ]:
# 문장을 예측한 emotion으로 변환
def predict(sentence):
    logits = test_sentences([sentence])
    label = np.argmax(logits)
    emotion = encoder.classes_[label]
    return emotion

In [ ]:
kaggle_dataset['Predicted'] = kaggle_dataset['utterance'].apply(predict)
kaggle_dataset.head()

,id,i_dialog,i_utterance,speaker,utterance,Predicted
0,0,0,0,Phoebe,"Alright, whadyou do with him?",neutral
1,1,0,1,Monica,Oh! You're awake!,surprise
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...,non-neutral
3,3,0,3,Mr. Tribbiani,"Yeah, but this is",neutral
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!,disgust


In [ ]:
temp_df = kaggle_dataset[['id', 'Predicted']]
temp_df = temp_df.rename(columns={'id': 'Id'})

In [ ]:
temp_df.to_csv('content/MyDrive/sentiment_analysis/friends_sample.csv', index=False)